# Visit the Wikipedia hyperlinks graph!

The goal of this project is to perform an analysis of the Wikipedia Hyperlink graph. 

In [12]:
import pandas as pd
import networkx as nx
import numpy as np
from collections import OrderedDict
from tqdm import tqdm

First of all we import the **wiki-topcats-reduced.txt** file where each row represents an edge (departure node and arrival one).

In [53]:
df = pd.read_csv('wiki-topcats-reduced.txt', sep = '\t', header = None)
df.columns = ['from', 'to']
df.head()

,from,to
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [15]:
# directed cause these two are different
print(df[df['from'] ==52], '\n')
print(df[df['from'] ==401135].head())

   from       to
0    52   401135
1    52  1069112
2    52  1163551 

          from      to
361754  401135   60219
361755  401135  167532
361756  401135  400980
361757  401135  401018
361758  401135  401019


By taking a look at the dataframe we notice that the graph is **directed**: in fact the edges are directed from one vertex to another. As we can see in the dataframe, a node could have edges departing from it but no edges to it. In the example presented above, we've seen that the node $52$ goes to the node $401135$ but the inverse edge is not present.

Since the graph is directed we cannot consider only one column of the dataframe thus the number of nodes is given by the length of the *set* of both columns while the number of edges is simply the length of the dataframe.

In [59]:
#vertices

fr = df.groupby('from').count()
to = df.groupby('to').count()
V = set(list(fr.index) + list(to.index))
print('Number of nodes =', len(V))
print('Number of edges:', len(df.index))


Number of nodes = 461193
Number of edges: 2645247


In [60]:
print('The average degree of the vertex is:', round(len(df.index)/len(V), 4))

The average degree of the vertex is: 5.7357


We have found that the average degree of all the vertexes is $5.74$ and this means that the graph is sparse. 
The average degree has been found with the formula:

$average\_degree$ = $\frac{total\_edges}{total\_vertex}$

Now we can work on the **wikipedia categories** and drop those with a number of articles less than $3500$ and create two dictionaries that have the same structure:
* categories = {category_name : nodes}

In [32]:
o = open('wiki-topcats-categories.txt', 'r')
categories = {}
categories_not_filtered = {}
for line in o :
    line = line.replace('\n','')
    l = line.split(' ')
    l[0] = l[0].replace('Category:', '').replace(';','')
    if len(l[1:]) >= 3500:
        categories[l[0]] = set(list(map(int, l[1:]))).intersection(set(V))
        categories_not_filtered[l[0]] =  set(list(map(int, l[1:])))

In [ ]:
categories, categories_not_filtered = category_creation()

In [82]:
o = open('wiki-topcats-page-names.txt', 'r')
names = {}
for line in o :
    line = line.replace('\n','')
    l = line.split(' ')
    names[int(l[0])] =  ' '.join(l[1:])

In [86]:
names

{0: 'Chiasmal syndrome',
 1: 'Kleroterion',
 2: 'Pinakion',
 3: 'LyndonHochschildSerre spectral sequence',
 4: "Zariski's main theorem",
 5: 'FultonHansen connectedness theorem',
 6: "Cayley's ruled cubic surface",
 7: 'Annulus theorem',
 8: "Bing's recognition theorem",
 9: 'BochnerMartinelli formula',
 10: 'BergmanWeil formula',
 11: 'Menallen Township, Pennsylvania',
 12: 'Missouri Route 117',
 13: 'Jadwin, Missouri',
 14: 'Gladden, Missouri',
 15: 'Missouri Route 119',
 16: 'Missouri Route 68',
 17: 'Sligo, Missouri',
 18: 'Lower Parker School',
 19: 'Lecoma, Missouri',
 20: 'Doss, Missouri',
 21: 'Boss, Missouri',
 22: 'Vulcan, Missouri',
 23: 'Glover, Missouri',
 24: 'Missouri Route 114',
 25: 'Missouri Route 12',
 26: 'Missouri Route 80',
 27: 'Missouri Route 75',
 28: 'Missouri Route 91',
 29: 'Missouri Route 147',
 30: 'Missouri Route 149',
 31: 'Goldsberry, Missouri',
 32: 'Missouri Route 162',
 33: 'Missouri Route 172',
 34: 'Missouri Route 245',
 35: 'Missouri Route 273',
 

In [91]:
dd = pd.DataFrame.from_dict(names, orient='index')

In [105]:
dd


,0
0,Chiasmal syndrome
1,Kleroterion
2,Pinakion
3,LyndonHochschildSerre spectral sequence
4,Zariski's main theorem
5,FultonHansen connectedness theorem
6,Cayley's ruled cubic surface
7,Annulus theorem
8,Bing's recognition theorem
9,BochnerMartinelli formula


The **categories** dictionary contains the nodes that have edges while **categories_not_filtered** contains all the nodes of each category.

Now it is asked to the user to insert an initial category.

In [19]:
input_category = input()

Year_of_birth_unknown


In [20]:
DG = nx.from_pandas_edgelist(df, 'from', 'to', create_using = nx.DiGraph )

In [10]:
class BFS:

    def bfs(self, dic):
        node = dic
        dist = 0
        l = list(self.graph[node])
        actual_visited = [node]
        while len(l) > 0:
            dist += 1 
            children = []
            for i in l:
                children += (self.graph[i])
                self.visited[i] += [dist]  
                actual_visited += ([i])
                
            l = set(children).difference(set(actual_visited))
                
    
    def __init__(self, graph, categories, input_category):
        
        self.graph = graph
        self.nodes = categories
        self.initial = input_category
        self.visited = {}
        for i in self.graph.nodes:
            self.visited[i] = []
        for i in tqdm(self.initial):
            self.visited[i] = [0]
            self.bfs(i)

        for i in self.initial:
                self.visited[i] = [0]

In order to find all the shortest paths from the input category to all the others, we have used the **BFS** algorithm. 
We have runned this algorithm for each node present in the initial category. At the end we have a dictionary that has as keys the nodes that have been visited with all the associated shortest paths.
An example of the final dictionary is shown here:

$\{node_i: [dist_1, dist_2,..., dist_n ]\}$ , where n is the number of nodes in the input category.

In [11]:
bfs = BFS(DG, categories, input_category)

100%|████████████████████████████████████████████████████████████████████████████| 2536/2536 [1:16:34<00:00,  1.81s/it]


All the process described above takes a long time, so, in order to save time, we have saved the data that can be accessed faster.

In [21]:
import json
with open('data.json', 'r') as fp:
    data = json.load(fp)

In [73]:
median = {}
for category in categories:
    if category != input_category:
        shortest_path={}
        sh=[]
        if len(set(categories_not_filtered[category]).intersection(set(list(map(int,data.keys())))))> 0:
            for node in set(categories_not_filtered[category]).intersection(set(list(map(int,data.keys())))):
                if len(data[str(node)]) > 0:
                    sh += (list(data[str(node)]))
            shortest_path[0] = sh
            shortest_path[1] = (len(categories[category])*len(categories[input_category]) - len(shortest_path))
            l = len(shortest_path[0]) + shortest_path[1]
            if shortest_path[1] >= int(l/2):
                median[category] = 10000
            else:
                median[category] = 1
            #median[category] = np.median(shortest_path)
        else:
            median[category] = 100**100
median[input_category] = -1

In [68]:
block_ranking = OrderedDict()
block_ranking = OrderedDict(sorted(median.items(), key=lambda x: x[1]))
block_ranking

OrderedDict([('Year_of_birth_unknown', -1),
             ('English_footballers', 10000),
             ('The_Football_League_players', 10000),
             ('Association_football_forwards', 10000),
             ('Association_football_goalkeepers', 10000),
             ('Association_football_midfielders', 10000),
             ('Association_football_defenders', 10000),
             ('Living_people', 10000),
             ('Harvard_University_alumni', 10000),
             ('Major_League_Baseball_pitchers', 10000),
             ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
              10000),
             ('Indian_films', 10000),
             ('Year_of_death_missing', 10000),
             ('English_cricketers', 10000),
             ('Year_of_birth_missing_(living_people)', 10000),
             ('Rivers_of_Romania', 10000),
             ('Main_Belt_asteroids', 10000),
             ('Asteroids_named_for_people', 10000),
             ('English-language_albums', 10000

The **articles** dictionary contains the articles and the corresponding categories in which it appears:
* articles = {article : list_of_categories}

In [48]:
# map each article to the categories it is present

articles = {}
for name in categories:
    for article in categories[name]:
        if article in articles:
            articles[article].append(name)
        else:
            articles[article] = [name]

It can happen that an article belongs to several categories: in this case we assign it to the input category if the input category is one of these otherwise we choose the category according to block_ranking, i.e. it will belong to the category closest to the input one.

In [49]:
# if an article belongs to multiple categories, choose one according to block_ranking

for article in articles:
    if len(articles[article]) > 1:
        minimum = ''
        for cat in articles[article]:
            if (minimum == '') or (block_ranking[minimum] > block_ranking[cat]):
                minimum = cat
        articles[article] = [minimum]

The dictionary **categories_after_ranking** has the category as key and a list of articles belonging to it.

In [50]:
categories_after_ranking = {}

for i in articles:  # article??
    if articles[i][0] in categories_after_ranking:
        categories_after_ranking[articles[i][0]] +=  [i]
    else:
        categories_after_ranking[articles[i][0]] =  [i]

Given a rank to all the categories, now it is time to assign a rank to all the nodes. This is possible working with subgraphs and weights of the edges.

We have followed these three steps:

*  Starting from the input category, C\0 we have computed its subgraph. Then the weights of all the edges departing from a node of the input category have been updated with the corresponding weight of that node.
* After this, we have computed the subgraph induced by the second category, according to the block ranking, that gives an initial weight to all the nodes in that category. All the weights are updated with those that belong to the in-edges that come from the input category.
* Now the step 2 is repeated for all the categories.

In [51]:
idx = 0
edges = nx.to_dict_of_lists(DG)

for name in tqdm(block_ranking):
    if idx == 0:
        weight_dict = {}
        subgraph = DG.subgraph(categories_after_ranking[name])
        weight_dict[name] = {}
        for node in (subgraph.in_degree):
            for arr in edges[node[0]]:
                    DG[node[0]][arr]['weight'] = node[1]
        idx = 1
        for  node in subgraph.in_degree:
            weight_dict[name][node[0]] = node[1]
    else:
        try:
            subgraph = DG.subgraph(categories_after_ranking[name])
            weight_dict[name] = {}

            for node in subgraph.in_degree:
                cumsum = node[1]
                for arr in edges[node[0]]:
                        try:
                            cumsum+=(list(DG.edges[arr,node[0]].values()))[0]
                        except:
                            pass
                weight_dict[name][node[0]] = cumsum


            for node in subgraph.in_degree:
                for arr in edges[node[0]]:
                        DG[node[0]][arr]['weight'] = weight_dict[name][node[0]]
        except:
            pass

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:19<00:00,  4.52it/s]


In [110]:
#rank per ogni nodo (come nello step3 dell'homework)
rank = []
for i in weight_dict:
    rank += list(OrderedDict(sorted(weight_dict[i].items(), key = lambda x: x[1], reverse = True)).keys())

ranks = {}
for i in range(50):
    ranks[rank[i]] = dd.loc[rank[i]].values[0]

In [128]:
pd.DataFrame.from_dict(ranks, orient='index', columns=['Article_name']).head(10)

,Article_name
62684,Diogenes Lartius
170163,Pausanias (geographer)
1656777,L Bu
1656780,Dong Zhuo
169696,Theocritus
1656794,Yuan Shu
170578,Stobaeus
1342864,Penda of Mercia
1343014,Symeon of Durham
1656778,Zhang Fei
